<a href="https://colab.research.google.com/github/JedWojtowicz/MAD_SWPS_Jedrzej_Wojtowicz_3/blob/main/MAD_JW_Projekt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
'''
#from google.colab import drive
#drive.mount('/content/drive')

'''

"\n#from google.colab import drive\n#drive.mount('/content/drive')\n\n"

In [15]:
import pandas as pd
csv_file = "/content/drive/MyDrive/metody analizy danych/Projekt/Laptop_price.csv"

In [16]:
#laptop: pd.DataFrame = pd.read_csv(csv_file, header=0)
laptop = pd.read_csv(csv_file)

In [17]:
display(laptop.head(10))

,Brand,Processor_Speed,RAM_Size,Storage_Capacity,Screen_Size,Weight,Price
0,Asus,3.830296,16,512,11.185147,2.641094,17395.093065
1,Acer,2.912833,4,1000,11.311372,3.260012,31607.605919
2,Lenovo,3.241627,4,256,11.853023,2.029061,9291.023542
3,Acer,3.806248,16,512,12.280360,4.573865,17436.728334
4,Acer,3.268097,32,1000,14.990877,4.193472,32917.990718
5,HP,1.881348,16,256,11.943958,4.840268,9543.719980
6,Lenovo,2.940721,32,256,11.713246,4.344846,10390.137319
7,Lenovo,3.016788,4,256,15.588315,2.941120,9214.484506
8,Lenovo,2.560327,4,512,15.026332,2.743620,16372.175284
9,Acer,3.341111,32,512,13.611945,4.093149,18501.832733


In [18]:
laptop.tail(5)

,Brand,Processor_Speed,RAM_Size,Storage_Capacity,Screen_Size,Weight,Price
995,HP,3.343584,4,1000,12.587095,3.162399,31593.668017
996,Dell,2.780555,8,256,12.679356,3.750265,9149.521832
997,Dell,3.200569,4,512,12.666315,3.392612,16552.404779
998,Asus,1.604182,8,256,11.215581,3.857613,9407.473459
999,Lenovo,1.711980,4,256,16.561498,3.440883,8807.696702


In [19]:
laptop.index

RangeIndex(start=0, stop=1000, step=1)

In [20]:
laptop.dtypes

,0
Brand,object
Processor_Speed,float64
RAM_Size,int64
Storage_Capacity,int64
Screen_Size,float64
Weight,float64
Price,float64


In [21]:
laptop.dtypes.value_counts()

,count
float64,4
int64,2
object,1


In [22]:
laptop.describe()

,Processor_Speed,RAM_Size,Storage_Capacity,Screen_Size,Weight,Price
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,2.750611,15.500000,584.576000,14.056790,3.466919,19604.187963
std,0.731796,10.988665,313.438517,1.705882,0.866541,9406.064880
min,1.511580,4.000000,256.000000,11.012111,2.000560,8570.012950
25%,2.089246,8.000000,256.000000,12.635523,2.717211,10114.012948
50%,2.760885,16.000000,512.000000,14.099643,3.464630,17287.241878
75%,3.362610,32.000000,1000.000000,15.528590,4.212583,31566.214754
max,3.998534,32.000000,1000.000000,16.985737,4.990728,33503.935037


In [23]:
col: pd.Series = laptop['Price']
col

,Price
0,17395.093065
1,31607.605919
2,9291.023542
3,17436.728334
4,32917.990718
...,...
995,31593.668017
996,9149.521832
997,16552.404779
998,9407.473459


In [24]:
laptop.select_dtypes(include=['float']).head()

,Processor_Speed,Screen_Size,Weight,Price
0,3.830296,11.185147,2.641094,17395.093065
1,2.912833,11.311372,3.260012,31607.605919
2,3.241627,11.853023,2.029061,9291.023542
3,3.806248,12.280360,4.573865,17436.728334
4,3.268097,14.990877,4.193472,32917.990718


In [25]:
laptop.select_dtypes(include=['int']).head()

,RAM_Size,Storage_Capacity
0,16,512
1,4,1000
2,4,256
3,16,512
4,32,1000


In [26]:
laptop.filter(like='Speed').head()

,Processor_Speed
0,3.830296
1,2.912833
2,3.241627
3,3.806248
4,3.268097


In [27]:
laptop['Price'].isnull().any()

False

In [29]:
laptop['Screen_Size'].head() > 14

,Screen_Size
0,False
1,False
2,False
3,False
4,True


In [30]:
laptop_new_idx: pd.DataFrame = laptop.set_index('Price')
laptop_new_idx

,Brand,Processor_Speed,RAM_Size,Storage_Capacity,Screen_Size,Weight
Price,,,,,,
17395.093065,Asus,3.830296,16,512,11.185147,2.641094
31607.605919,Acer,2.912833,4,1000,11.311372,3.260012
9291.023542,Lenovo,3.241627,4,256,11.853023,2.029061
17436.728334,Acer,3.806248,16,512,12.280360,4.573865
32917.990718,Acer,3.268097,32,1000,14.990877,4.193472
...,...,...,...,...,...,...
31593.668017,HP,3.343584,4,1000,12.587095,3.162399
9149.521832,Dell,2.780555,8,256,12.679356,3.750265
16552.404779,Dell,3.200569,4,512,12.666315,3.392612
